In [1]:
import setup, os, json
from recordings.audio_file_path import audio_file_path,transcript_name

transcribing file: 
/Users/ayserchoudhury/playground/jarvis/transcribe/recordings/sample.wav


In [2]:
# Setting up transcriber
from transformers import pipeline
print("initiating transcription pipeline..")
transcriber = pipeline("automatic-speech-recognition",model="openai/whisper-large-v3",max_new_tokens=100, chunk_length_s=30)
print("transcription pipelines ready.")

/Users/ayserchoudhury/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


initiating transcription pipeline..


In [ ]:
# Transcribing full audio with timestamps
print("transcribing audio...")
full_transcription = transcriber(audio_file_path,max_new_tokens=256,generate_kwargs={"task": "transcribe"},chunk_length_s=30,return_timestamps=True)
print("completed transcription.")

# print(full_transcription)

# chunks = full_transcription["chunks"]
# for c in chunks:
#     print(c)

full_transcription["chunks"][1]["timestamp"][0]

In [3]:
# Setting up diarizer 
from pyannote.audio import Pipeline
from keys import pyannote_key

print("initializing diarization pipeline...")
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=pyannote_key)
print("diarization pipeline ready.")
print("diarizing audio..")
diarization = pipeline(audio_file_path)

d_starts = []
d_ends = []
d_speakers = []

for segment, _, speaker in diarization.itertracks(yield_label=True):
    d_starts.append(segment.start)
    d_ends.append(segment.end)
    d_speakers.append(speaker)
    
print("completed diarization.")

/Users/ayserchoudhury/anaconda3/lib/python3.11/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


initializing diarization pipeline...
diarizing audio..
completed diarization.


In [4]:
print(d_speakers)
print(d_starts)
print(d_ends)

['SPEAKER_00', 'SPEAKER_01', 'SPEAKER_00', 'SPEAKER_00', 'SPEAKER_01', 'SPEAKER_00', 'SPEAKER_01', 'SPEAKER_00', 'SPEAKER_00', 'SPEAKER_01', 'SPEAKER_00', 'SPEAKER_01']
[0.6536502546689305, 4.473684210526315, 7.427843803056028, 9.872665534804755, 11.621392190152802, 14.779286926994908, 20.551782682512737, 21.7572156196944, 24.01528013582343, 26.137521222410868, 28.599320882852297, 33.74363327674024]
[4.1341256366723265, 6.867572156196944, 9.617996604414262, 11.451612903225808, 13.964346349745332, 20.28013582342954, 23.132427843803057, 21.7911714770798, 25.916808149405774, 28.07300509337861, 33.217317487266556, 36.44312393887946]


In [5]:
import librosa
import numpy as np
y, sr = librosa.load(audio_file_path)

# Convert to desired data type (float32 or float64)
audio_array = y.astype(np.float32)  # or np.float64

# Ensure it's 1D array
if audio_array.ndim > 1:
    audio_array = audio_array.flatten()

print(f"Shape: {audio_array.shape}, Data type: {audio_array.dtype}, Sampling rate: {}")

Shape: (860803,), Data type: float32, Sampling rate: 22050


In [7]:
print(len(audio_array))

# multiplier = len(audio_array)/d_ends[len(d_ends)-1]
print(audio_array[:5])
print(d_ends[len(d_ends)-1])

860803
[ 1.1343518e-06 -1.8206911e-06 -2.1902491e-05 -2.0330670e-05
 -1.6933725e-05]
36.44312393887946


In [8]:
processed_data = np.array(audio_array).astype(np.float32) / 32767.0
print(len(processed_data))
print(processed_data[:5])
print(type(processed_data))

860803
[ 3.4618728e-11 -5.5564778e-11 -6.6843137e-10 -6.2046174e-10
 -5.1679205e-10]
<class 'numpy.ndarray'>


In [23]:
transcriber({"sampling_rate": sr, "raw": processed_data})

{'text': ' you you'}

{'text': ' you you'}

In [ ]:
x = 0
print(multiplier)

diarized_transcription = ""
diarized_transcription += d_speakers[i]+ ": "+ transcription['text'] + "\n"

print(len(audio_array))
for i in range(0,len(d_ends)):
    print(x)
    if (int(d_ends[i] * multiplier)>len(audio_array)):
        transcription = transcriber(audio_array[x:len(audio_array)-1],generate_kwargs={"task": "transcribe"})
        diarized_transcription += d_speakers[i]+ ": "+ transcription['text'] + "\n"
    else:
        y = int(d_ends[i] * multiplier)
        transcription = transcriber(audio_array[x:y],generate_kwargs={"task": "transcribe"})
    diarized_transcription += d_speakers[i]+ ": "+ transcription['text'] + "\n"
    x = y
    
    


In [ ]:
diarized_transcription

In [ ]:
from pydub import AudioSegment

# Load audio file
audio = AudioSegment.from_wav(audio_file_path)
print(len(audio))
diarized_transcription = ""

# Split and save chunks
for i, timestamp in enumerate(d_ends):
    start, end = timestamp, d_ends[i + 1] if i + 1 < len(d_ends) else None
#     print(start)
#     print(end)
    segment = audio[start:end]
    print(end-start)
#     segment.export(f"segment_{i + 1}.wav", format="wav")
    transcription = transcriber(f"segment_{i + 1}.wav",max_new_tokens=256,generate_kwargs={"task": "transcribe"},chunk_length_s=30,return_timestamps=True)
    diarized_transcription += d_speakers[i]+ ": "+ transcription['text'] + "\n"
#     print(transcription)

In [ ]:
print(diarized_transcription)

In [ ]:
import librosa
import numpy as np

# Load audio
y, sr = librosa.load(audio_file_path)
diarized_transcription = ""

# Define timestamps (in seconds)
timestamps = d_ends

# Convert timestamps to sample indices
sample_indices = [int(timestamp * sr) for timestamp in timestamps]

# Split and save chunks
for i, start_index in enumerate(sample_indices):
    end_index = sample_indices[i + 1] if i + 1 < len(sample_indices) else len(y)
    audio_chunk = y[start_index:end_index]
    transcription = transcriber(audio_chunk,max_new_tokens=256,generate_kwargs={"task": "transcribe"},chunk_length_s=30,return_timestamps=True)
    diarized_transcription += d_speakers[i]+ ": "+ transcription['text'] + "\n"
    # Save or process the audio chunk using appropriate methods

In [ ]:
print(diarized_transcription)